In [79]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
from implicit.als import AlternatingLeastSquares
import scipy.sparse as sparse


In [43]:
read_file = pd.read_csv (r'C:\Users\Johan\Downloads\dataset_TIST2015\dataset_TIST2015_POIs.txt', names=["Pid", "Latitude", "Longitude", "Category", "Country"], sep='\t')
read_file.to_csv (r'C:\Users\Johan\Documents\POI_dataset\TIST2015_POIs.csv', index=None)

In [ ]:
read_file = pd.read_csv (r'C:\Users\Johan\Downloads\dataset_TIST2015\dataset_TIST2015_Checkins.txt', names=["Uid","Pid","Timestamp", "TimezoneOffset"], sep='\t')
read_file.to_csv (r'C:\Users\Johan\Documents\POI_dataset\TIST2015_Checkins.csv', index=None)

In [44]:
def read_data(path):
    files = {}
    for filename in path.glob('*'):
        if filename.suffix == '.csv':
            files[filename.stem] = pd.read_csv(filename)
        elif filename.suffix == '.txt':
            if filename.stem == "TIST2015_POIs":
                columns = ["Pid", "Latitude", "Longtitude", "Category", "Country"]
            elif filename.stem == "TIST2015_POIs_Checkins":
                columns = ["Uid","Pid","Timestamp", "TimezoneOffset"]
            else:
                continue
            data = pd.read_csv(filename, names=columns, engine='python')
            files[filename.stem] = data

    return files['TIST2015_POIs'], files['TIST2015_Checkins']

In [132]:
# pick one of the available folders
pois, checkins = read_data(Path(r'C:\Users\Johan\Documents\POI_dataset'))

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [131]:
pois

,Pid,Latitude,Longitude,Category,Country,pid
0,3fd66200f964a52000e71ee3,40.733596,-74.003139,Jazz Club,US,3fd66200f964a52019ec1ee3
1,3fd66200f964a52000e81ee3,40.758102,-73.975734,Gym,US,3fd66200f964a52019ec1ee3
2,3fd66200f964a52000ea1ee3,40.732456,-74.003755,Indian Restaurant,US,3fd66200f964a52019ec1ee3
3,3fd66200f964a52000ec1ee3,42.345907,-71.087001,Indian Restaurant,US,3fd66200f964a52019ec1ee3
4,3fd66200f964a52000ee1ee3,39.933178,-75.159262,Sandwich Place,US,3fd66200f964a52019ec1ee3
...,...,...,...,...,...,...
3680121,5237865c498e89110c1d03e7,40.154444,26.410847,Comedy Club,TR,3fd66200f964a52019ec1ee3
3680122,5237867411d2a1e910744c81,35.340099,33.309328,Home (private),CY,3fd66200f964a52019ec1ee3
3680123,5237879111d216bab10e9e09,-1.404065,-48.453742,Home (private),BR,3fd66200f964a52019ec1ee3
3680124,52378c24498ea9502baf2716,3.425155,-76.545010,Sandwich Place,CO,3fd66200f964a52019ec1ee3


In [ ]:
users_subset = user_checkins_counts['Uid'].unique()
len(users_subset)
checkins.sample(users_subset, 1000)
##users_subset=checkins.loc[checkins['Uid'] <= 1000, 'Pid']
##users_subset


##checkins.head()

In [ ]:
checkins["Uid"].plot(kind='hist')


In [ ]:
checkins.describe(include=object)

In [ ]:
checkins["Uid"].value_counts()


In [ ]:
user_checkins_counts = checkins.groupby('Uid')['Pid'].count()

In [46]:
# Create a subset of a dataset with size n users
def create_checkin_subset(checkins_table, n):
    subset = np.array([[0,0,0,0],[0,0,0,0]])
    checkins_100_users = checkins_table['Uid'].sample(n=n, random_state=1)
    for i in checkins_100_users:
        subset = np.append(subset, np.asarray(checkins_table[checkins_table['Uid'] == i].values), 0)
    return pd.DataFrame(subset[2::], columns=['Uid', 'Pid', 'Timestamp', 'TimezoneOffset'])

In [83]:
# Initialize the subset method
checkins_100_users = create_checkin_subset(checkins, 50)

In [ ]:
read_file = pd.DataFrame(checkins_100_users)
read_file.to_csv (r'C:\Users\Johan\Documents\POI_dataset\TIST2015_Checkins_50.csv', index=None)

In [84]:
checkins_100_users
##points_checkins_counts = checkins.groupby('Pid')['Uid'].count()

,Uid,Pid,Timestamp,TimezoneOffset
0,33634,4ef5d479f9ab2e6680a7fdda,Sun Feb 17 09:58:34 +0000 2013,120
1,33634,4e621c8d62e13e3bce7bb58d,Sun Feb 17 11:17:48 +0000 2013,120
2,33634,4cb0fb5af2dbef3b6aae79e5,Mon Feb 18 06:34:41 +0000 2013,120
3,33634,5110b08de4b0d12c384d4e0b,Mon Feb 18 06:38:31 +0000 2013,120
4,33634,4d446c883616b60c0435e4c2,Mon Feb 18 06:50:21 +0000 2013,120
...,...,...,...,...
15092,147721,4fdca198e4b0598b20b0040e,Mon May 13 10:30:44 +0000 2013,420
15093,147721,50fa70f4e4b022d513e5ea45,Sun May 19 10:01:05 +0000 2013,420
15094,147721,4e76935dfa76059701857f95,Thu May 30 11:16:06 +0000 2013,420
15095,147721,512c9a39e4b0c161e0b39cc0,Wed Jun 26 07:45:26 +0000 2013,420


In [86]:
sparse_item_user = sparse.csr_matrix((checkins_100_users['TimezoneOffset'], (checkins_100_users['Pid'], checkins_100_users['Uid'])))
sparse_user_item = sparse.csr_matrix((checkins_100_users['TimezoneOffset'], (checkins_100_users['Uid'], checkins_100_users['Pid'])))

TypeError: 'str' object cannot be interpreted as an integer

In [87]:
# Merge pois and checkins to one Dataframe
checkin_data = checkins_100_users.merge(pois, on='Pid')

In [88]:
# create a matrix of Uid and Pid
df = checkin_data.set_index('Uid').Pid.str.get_dummies(',')
df = df.groupby('Uid').max()


In [50]:
listofusers = pd.DataFrame(checkin_data, columns= ['Uid']).groupby('Uid').max().sample(frac=1)
listofpois = pd.DataFrame(checkin_data, columns= ['Pid', 'Latitude', 'Longitude']).groupby('Pid').max().sample(frac=1)

userarray = listofusers.index.to_numpy()
poiarray = listofpois.index.to_numpy()

userdataframe = pd.DataFrame(userarray, columns = ['Users'])
poidataframe = pd.DataFrame(poiarray, columns = ['Poi'])

In [42]:
listofpois

,Latitude,Longitude
Pid,,
4c6aff4f23971b8df85d998a,-6.905679,NaN
510e4428e4b0f2f1ac068c23,-6.175188,NaN
4f47639fe4b08df2fa50958c,55.782112,NaN
441130a9f964a520c6301fe3,30.391664,NaN
50fad1cae4b011742849fd89,41.085380,NaN
...,...,...
4b7d0095f964a5209fab2fe3,-6.912624,NaN
4ebf52cbc2ee23891e42f852,-6.137852,NaN
4ba9c796f964a520c5373ae3,55.692064,NaN


In [51]:
dot = userdataframe.merge(poidataframe, how='cross')

In [52]:
dot

,Users,Poi
0,71607,4a32df52f964a520ad9a1fe3
1,71607,4f38bb68e4b0b0eb8e887ec8
2,71607,4d84e9860a0d1456c7a307cc
3,71607,4b70652af964a520b7172de3
4,71607,4cd53da467adf04d77de2b91
...,...,...
254295,220716,4e208d05c65b2b5e353d4c53
254296,220716,4e640e7a62e13e3bcf5895dc
254297,220716,4b16d1e2f964a520e9bd23e3
254298,220716,4b6cb20cf964a520ea4c2ce3


In [53]:
rows_list = []
for i in range(len(dot)):
    temp = df[dot.loc[i, "Poi"]][dot.loc[i, "Users"]]
    dict1 = {'ground_truth':float(temp)}
    rows_list.append(dict1)

groundtruth = pd.DataFrame(rows_list)

In [54]:
result = pd.concat([dot, groundtruth], axis=1)


In [55]:
rows_list = []
for i in range(len(dot)):
    temp = dot.loc[i, "Poi"]
    latitude = listofpois.loc[temp]['Latitude']
    longitude = listofpois.loc[temp]['Longitude']
    dict1 = {'Latitude':latitude, 'Longitude':longitude}
    rows_list.append(dict1)
    #latitude = poiarray[i]
latlong = pd.DataFrame(rows_list)

In [56]:
latlong

,Latitude,Longitude
0,41.903229,-87.680921
1,3.127236,101.723491
2,41.054211,28.933804
3,52.339692,5.132638
4,-6.178870,106.792688
...,...,...
254295,41.075079,28.999899
254296,-15.809471,-48.059026
254297,33.500979,-111.918717
254298,-29.990475,-51.177492


In [ ]:
checkin_data

In [57]:
userdot = pd.DataFrame(dot, columns= ['Users'])
latlong['Latitude'] = pd.to_numeric(latlong['Latitude'])
latlong['Longitude'] = pd.to_numeric(latlong['Longitude'])
dataset = pd.concat([userdot, latlong], axis=1)

In [58]:
dataset

,Users,Latitude,Longitude
0,71607,41.903229,-87.680921
1,71607,3.127236,101.723491
2,71607,41.054211,28.933804
3,71607,52.339692,5.132638
4,71607,-6.178870,106.792688
...,...,...,...
254295,220716,41.075079,28.999899
254296,220716,-15.809471,-48.059026
254297,220716,33.500979,-111.918717
254298,220716,-29.990475,-51.177492


In [ ]:
df.stack().reset_index().rename(columns={'Uid': 'user_id', 'Pid': 'poi_id', 0:'label'})

In [161]:
df.T

Uid,200,546,738,1688,2433,6965,8366,8828,10918,11081,...,147721,148340,151437,159875,160362,183523,211392,220716,223186,235013
3fd66200f964a52019ec1ee3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3fd66200f964a5202fec1ee3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3fd66200f964a5203ff11ee3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3fd66200f964a52058e41ee3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3fd66200f964a52058ec1ee3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5218f8bd11d276315afde5c2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
521b38c311d28ce840936f2f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5221e8c711d29e4ef04c90e3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
522304ba11d2fdf82198e013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [121]:
Pid_list = list(df.columns.values)
Uid_list = list(df.index.values)

user_item_list = Pid_list

5086

In [165]:
import scipy

dftomatrix = scipy.sparse.csr_matrix(df.values)

In [163]:
sdf = pd.DataFrame.sparse.from_spmatrix(dftomatrix)

In [164]:
sdf.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(factors=64, regularization=0.05)
model.fit(2 * dftomatrix)

  0%|          | 0/15 [00:00<?, ?it/s]

In [198]:
userid = 1
ids, scores = model.recommend(userid, dftomatrix[userid], N=100, filter_already_liked_items=False)

In [168]:
df = df.T

AttributeError: 'int' object has no attribute 'values'

In [203]:
l = []
l2 = []
for i in df[Uid_list[userid]]:
    if i == 1:
        l.append(i)
    else:
        l2.append(i)
print("length of 1's ", len(l))
print("length of 0's ", len(l2))


length of 1's  365
length of 0's  4721


In [212]:
# Use pandas to display the output in a table, pandas isn't a dependency of implicit otherwise

pd.DataFrame({"points": [Pid_list[i] for i in ids], "score": scores, "already_visited": np.in1d(ids, checkins_100_users['Uid'][Uid_list[userid]])})

,points,score,already_visited
0,4bddae1f89ca76b09b7d5d5e,0.999677,False
1,4b5086ccf964a520212627e3,0.999677,False
2,4b63406af964a520a36d2ae3,0.999622,False
3,4bdd5f27f219c9b6bb751210,0.999622,False
4,4e68694845dd6cc5bcfca67d,0.999611,False
...,...,...,...
95,4d5c0f3f2a31a1cd8fd443f1,0.999611,False
96,4d3719620a0b721ebbd7979b,0.999611,False
97,4c2ce3abd1a10f47712dfa64,0.999611,False
98,4c1448d9f1e0b7133dcc34bc,0.999611,False


In [ ]:
pd.DataFrame(df)

In [ ]:
# Remove Uid Duplicates in matrix
df = df.groupby('Uid').max()
print(df)

In [ ]:
Users = checkins['Uid'].unique()
print(len(Users))
Ncheckins = np.zeros(len(Users))
for user,i in zip(Users, range(len(Users))):
    Ncheckins[i] = np.where(checkins['Uid'] == Users)

In [ ]:
plt.hist(user_checkins_counts)
plt.show()
##user_checkins_counts.plot(kind='hist', x=user_checkins_counts, y=points_checkins_counts)

In [ ]:
def tabular_preview(checkins, n=15):
    """Creates a cross-tabular view of users vs movies."""

    user_groups = checkins.groupby('Uid')['Pid'].count()
    top_users = user_groups.sort_values(ascending=False)[:15]

    poi_groups = checkins.groupby('Pid')['Uid'].count()
    top_pois = poi_groups.sort_values(ascending=False)[:15]

    top = (
        checkins.
        join(top_users, rsuffix='_r', how='inner', on='Uid').
        join(top_pois, rsuffix='_r', how='inner', on='Pid'))

    plt.hist(top)
    plt.show()

In [ ]:
tabular_preview(checkin_data)

In [ ]:
A=nx.linalg.graphmatrix.adjacency_matrix(checkins_100_users).todense()
L=nx.linalg.laplacianmatrix.laplacian_matrix(checkins_100_users).todense()

In [ ]:
A=

